In [1]:
# data engineer
import pandas as pd
import numpy as np
import queue
from pprint import pprint
from sklearn.ensemble import IsolationForest
import math
from tqdm import tqdm
from datetime import datetime, timedelta
FILE_FOLDER = '../prediction_data/%s'
AFTER_FILE_FOLDER = '../prediction_data/afterAnalysis/%s'

In [3]:
def point_dis_square(x1, y1, x2, y2):
    dis = (x1 * 1.0 - x2) * (x1 * 1.0 - x2) + (y1 * 1.0 - y2) * (y1 * 1.0 - y2)
    return math.sqrt(dis)

def get_nearest_stations(df_grid_stations,closest_num=10):
    df_aq_stations = pd.read_csv(AFTER_FILE_FOLDER % 'aq_stations.csv')
#     df_grid_stations = pd.read_csv(AFTER_FILE_FOLDER % 'grid_stations.csv')

    aq_len = len(df_aq_stations.stationId)
    grid_len = len(df_grid_stations)

#     closest_num = 10
    aq_closest_dict = {}

    for i in range(aq_len):

        aq_stationId = df_aq_stations.stationId[i]

        aq_la = df_aq_stations.latitude[i]
        aq_lo = df_aq_stations.longitude[i]
        aq_closest_dict[aq_stationId] = []
        aq_q = queue.PriorityQueue()

        for j in range(grid_len):
            grid_la = df_grid_stations.latitude[j]
            grid_lo = df_grid_stations.longitude[j]
            try:
                stationId = df_grid_stations.gridStationId[j]
            except:
                stationId = df_grid_stations.stationId[j]

            y = (point_dis_square(aq_la, aq_lo, grid_la, grid_lo))
            if (aq_q.qsize() < closest_num):
                aq_q.put((-1.0 * y, stationId))
            else:
                if (aq_q.queue[0][0] < -1.0 * y):
                    aq_q.get()
                    aq_q.put((-1.0 * y, stationId))

        while (not aq_q.empty()):
            current_ele = aq_q.get()
            aq_closest_dict[aq_stationId].append((-1*current_ele[0],current_ele[1]))
        # pprint(aq_closest_dict[aq_stationId])

#     return aq_closest_dict,df_aq_stations
    return aq_closest_dict

In [22]:
def get_mean(df, feature_name):
    df_nearest_3 = df
    weight_sum = df_nearest_3['weight'].sum()
    df_nearest_3['weight'] = df_nearest_3['weight'] / weight_sum
    df_nearest_3['after_weighted'] = df_nearest_3['weight'] * df_sub[feature_name]
    return np.around(df_nearest_3['after_weighted'].sum(), decimals=2)

def fill_new_aqStations(nearest_ow_dict,df_new,df_ow):
    error_num = 0
    df_new_copy = df_new.copy()
    df_new_copy = df_new_copy.set_index(['aq_station_id','time'])
    
    df_ow_new = df_ow.set_index(['station_id','time'])
    df_null = df_new[df_new.isnull().any(axis=1)]
    station_list = df_null['aq_station_id'].unique().tolist()
    df_null = df_null.set_index(['aq_station_id'])
    for station in station_list:
        df_station = df_null.loc[station]
        df_ow_cu = df_ow_new.loc[nearest_ow_dict[station][0][1]]

        for index, row in df_station.iterrows():
            try:
                ow =  df_ow_cu.loc[row['time']]
#                 print(ow)
                for item in ['temperature','pressure','humidity','wind_speed']:
#                     df_new_copy.set_value((station,row['time']),item,ow[item])
                    df_new_copy.at[(station,row['time']),item] = ow[item]
            except:
                error_num+=1
                for item in ['temperature','pressure','humidity','wind_speed']:
                    last_hour_date_time = datetime.strptime(row['time'],'%Y-%m-%d %H:%M:%S') - timedelta(hours = 1)
                    last_hour = last_hour_date_time.strftime('%Y-%m-%d %H:%M:%S')
                    df_new_copy.at[(station,row['time']),item] = df_new_copy.at[(station,last_hour),item]
#         break
    print(error_num)
    return df_new_copy.reset_index()

# ss = fill_new_aqStations(nearest_ow_dict,df_new04,df_ow_stations_04)

# ss.to_csv(AFTER_FILE_FOLDER % 'new_f04_no_missing.csv',index=False)

In [13]:
df_ow_stations_1701_1801 = pd.read_csv(FILE_FOLDER%'observedWeather_201701-201801.csv')
print(df_ow_stations_1701_1801.columns)
df_ow_stations_1701_1801['datetime'] = pd.to_datetime(df_ow_stations_1701_1801['utc_time'])
df_ow_stations_1801 = df_ow_stations_1701_1801[
    df_ow_stations_1701_1801['datetime'] >='2018-01-01 00:00:00'
]

df_ow_stations_02_03 = pd.read_csv(FILE_FOLDER%'observedWeather_201802-201803.csv')
df_ow_stations_02_03['datetime'] = pd.to_datetime(df_ow_stations_02_03['utc_time'])

df_ow_stations_02_03.columns = ['station_id', 'time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed', 'wind_direction', 'datetime']
df_ow_stations_1801.columns = ['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'weather',
       'datetime']
selected_features = ['station_id', 'temperature', 'pressure','humidity', 'wind_speed', 'time', 'datetime']
df_ow_stations_02_03 = df_ow_stations_02_03[selected_features]
print(df_ow_stations_02_03.columns)
df_ow_stations_1801 = df_ow_stations_1801[selected_features]
print(df_ow_stations_1801.columns)
df_ow_0103 = df_ow_stations_1801.append(df_ow_stations_02_03)
df_ow_0103.reset_index(inplace = True)
df_ow_0103 = df_ow_0103[selected_features]
df_ow_0103.head()

Index(['station_id', 'longitude', 'latitude', 'utc_time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'weather'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')


,station_id,temperature,pressure,humidity,wind_speed,time,datetime
0,shunyi_meo,-5.2,1025.1,36,2.0,2018-01-01 00:00:00,2018-01-01 00:00:00
1,shunyi_meo,-2.2,1026.0,28,2.1,2018-01-01 01:00:00,2018-01-01 01:00:00
2,shunyi_meo,-0.6,1026.6,23,2.7,2018-01-01 02:00:00,2018-01-01 02:00:00
3,shunyi_meo,1.0,1026.1,20,2.2,2018-01-01 03:00:00,2018-01-01 03:00:00
4,shunyi_meo,1.9,1024.9,16,3.6,2018-01-01 04:00:00,2018-01-01 04:00:00


In [14]:
df_new0103 = pd.read_csv(AFTER_FILE_FOLDER % 'new_f0103.csv')
df_ow_stations = pd.read_csv(AFTER_FILE_FOLDER % 'ob_weather_stations.csv')

nearest_ow_dict = get_nearest_stations(df_ow_stations, 1)

In [23]:
df_after = fill_new_aqStations(nearest_ow_dict,df_new0103,df_ow_0103)

140


In [24]:
# df_after[df_after.isnull().any(axis=1)]
df_after.to_csv(AFTER_FILE_FOLDER % 'new_f0103_no_missing.csv',index=False)

In [83]:
nearest_ow_dict.keys()

dict_keys(['dongsi_aq', 'tiantan_aq', 'guanyuan_aq', 'wanshouxigong_aq', 'aotizhongxin_aq', 'nongzhanguan_aq', 'wanliu_aq', 'beibuxinqu_aq', 'zhiwuyuan_aq', 'fengtaihuayuan_aq', 'yungang_aq', 'gucheng_aq', 'fangshan_aq', 'daxing_aq', 'yizhuang_aq', 'tongzhou_aq', 'shunyi_aq', 'pingchang_aq', 'mentougou_aq', 'pinggu_aq', 'huairou_aq', 'miyun_aq', 'yanqin_aq', 'dingling_aq', 'badaling_aq', 'miyunshuiku_aq', 'donggaocun_aq', 'yongledian_aq', 'yufa_aq', 'liulihe_aq', 'qianmen_aq', 'yongdingmennei_aq', 'xizhimenbei_aq', 'nansanhuan_aq', 'dongsihuan_aq'])

In [75]:
last_hour_date_time = datetime.strptime('2018-04-01 16:00:00','%Y-%m-%d %H:%M:%S') - timedelta(hours = 1)
type(last_hour_date_time.strftime('%Y-%m-%d %H:%M:%S'))

str